OPEN AI_CLIP GIT
https://github.com/openai/CLIP/blob/main/notebooks/Interacting_with_CLIP.ipynb

In [1]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

     ---------------------------------------- 0.0/53.1 kB ? eta -:--:--
     ------- -------------------------------- 10.2/53.1 kB ? eta -:--:--
     --------------------- ---------------- 30.7/53.1 kB 262.6 kB/s eta 0:00:01
     -------------------------------------- 53.1/53.1 kB 455.9 kB/s eta 0:00:00
  Obtaining dependency information for regex from https://files.pythonhosted.org/packages/fc/85/0d1038f068900896a8590d6d0da198b90d31f731a39166a432aa2b92249b/regex-2023.10.3-cp39-cp39-win_amd64.whl.metadata
  Using cached regex-2023.10.3-cp39-cp39-win_amd64.whl.metadata (41 kB)
Using cached regex-2023.10.3-cp39-cp39-win_amd64.whl (269 kB)
  Cloning https://github.com/openai/CLIP.git to c:\users\pari0\appdata\local\temp\pip-req-build-2jh06afn
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for clip: filename=clip-1.0-py3-non

  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git 'C:\Users\pari0\AppData\Local\Temp\pip-req-build-2jh06afn'


In [20]:
import os
import clip
import torch
from torch import nn, optim

from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.datasets import ImageFolder

%matplotlib inline
BATCH_SIZE = 32
EPOCH = 5


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# Prepare the Model and Data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
print(device)

None


In [27]:
model, preprocess = clip.load("ViT-B/32", device=device, jit=False)
model = model.to(torch.float32)
# model.half()

In [28]:
# 이미지와 텍스트 리스트로 만들어서 확인
# 텍스트 tokenizer하기 전, Mydataset에 구현 되어있음

data_folder = 'mushroom_data_new/mushroom_data_new'

image_paths = []
text_descriptions = []

class_folders = os.listdir(data_folder)

for class_folder in class_folders:
    class_folder_path = os.path.join(data_folder, class_folder)
    image_files = os.listdir(class_folder_path)

    for image_file in image_files:
        image_path = os.path.join(class_folder_path, image_file)
        image_paths.append(image_path)

        # Create text description using class label
        text_description = f"a photo of {class_folder.replace('_',' ')}"
        text_descriptions.append(text_description)

print(text_descriptions)

['a photo of almond mushroom', 'a photo of almond mushroom', 'a photo of almond mushroom', 'a photo of almond mushroom', 'a photo of almond mushroom', 'a photo of almond mushroom', 'a photo of almond mushroom', 'a photo of almond mushroom', 'a photo of almond mushroom', 'a photo of almond mushroom', 'a photo of almond mushroom', 'a photo of almond mushroom', 'a photo of almond mushroom', 'a photo of almond mushroom', 'a photo of almond mushroom', 'a photo of almond mushroom', 'a photo of almond mushroom', 'a photo of almond mushroom', 'a photo of almond mushroom', 'a photo of almond mushroom', 'a photo of almond mushroom', 'a photo of almond mushroom', 'a photo of almond mushroom', 'a photo of almond mushroom', 'a photo of almond mushroom', 'a photo of almond mushroom', 'a photo of almond mushroom', 'a photo of almond mushroom', 'a photo of almond mushroom', 'a photo of almond mushroom', 'a photo of almond mushroom', 'a photo of almond mushroom', 'a photo of almond mushroom', 'a photo 

In [29]:
class MyDataset(Dataset):
    def __init__(self, data_folder, preprocess):
        self.data_folder = data_folder
        self.preprocess = preprocess
        self.image_paths = []
        self.text_descriptions = []

        # image path와 text list 형성
        class_folders = os.listdir(data_folder)

        for class_folder in class_folders:
            class_folder_path = os.path.join(data_folder, class_folder)
            image_files = os.listdir(class_folder_path)

            for image_file in image_files:
                image_path = os.path.join(class_folder_path, image_file)
                self.image_paths.append(image_path)

                # Create text description using class label
                text_description = clip.tokenize(f"a photo of {class_folder.replace('_',' ')}")
                self.text_descriptions.append(text_description)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx])
        image = self.preprocess(image)
        text = self.text_descriptions[idx]
        return image, text

# 데이터 전처리 및 데이터 로더 생성
data_folder = 'mushroom_data_new/mushroom_data_new'
dataset = MyDataset(data_folder, preprocess)

# 데이터 분할 (train, validation, test)
total_size = len(dataset)
train_size = int(0.7 * total_size)
val_size = int(0.2 * total_size)
test_size = total_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# 데이터 로더 생성
trainloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
testloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [30]:
from numpy.lib import shape_base
for batch in trainloader:
    images, texts = batch
    # 첫 번째 배치만 출력합니다.
    print("Image Path:", images[0].shape)
    # print("Label:", labels[0])
    print("Text Description:", texts[0].shape)
    break

Image Path: torch.Size([3, 224, 224])
Text Description: torch.Size([1, 77])


# Building Features

In [ ]:
# image_inputs = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
# text_inputs = torch.cat([clip.tokenize(f"a photo of a {c}") for c in dataset.classes]).to(device)

# image_inputs.shape, text_inputs.shape

# Training

In [44]:
def convert_models_to_fp32(model):
    for p in model.parameters():
        p.data = p.data.float()
        p.grad.data = p.grad.data.float()

loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-5)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, len(trainloader)*EPOCH)

In [ ]:
# # 분류를 위한 추가적인 레이어 설정
# classifier = torch.nn.Linear(512, 215).to(device)

# # 손실 함수 및 옵티마이저 설정
# criterion = torch.nn.CrossEntropyLoss()
# optimizer = optim.Adam(classifier.parameters(), lr=0.001)

# # 모델 파인튜닝
# model.eval()
# for epoch in range(EPOCH):  # 에폭 수를 조정하세요
#     classifier.train()
#     running_loss = 0.0
#     for i, batch in enumerate(trainloader, 0):
#         images, texts, labels = batch
#         images = images.to(device)
#         labels = labels.to(device)

#         optimizer.zero_grad()

#         with torch.no_grad():
#             text_inputs = clip.tokenize(texts).to(device)
#             text_features = model.encode_text(text_inputs)

#         image_features = model.encode_image(images)
#         text_features /= text_features.norm(dim=-1, keepdim=True)
#         image_features /= image_features.norm(dim=-1, keepdim=True)

#         # 이미지 특성을 사용하여 예측 (데이터 타입 일치)
#         logits = classifier(image_features.to(classifier.weight.dtype))

#         loss = criterion(logits, labels)
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()
#         if i % 10 == 9:
#             print(f"[Epoch {epoch + 1}, Batch {i + 1}] Loss: {running_loss / 10:.3f}")
#             running_loss = 0.0

# print("Finished Fine-tuning")

In [ ]:
from tqdm.notebook import tqdm

for epoch in range(EPOCH):
    print(f"Epoch: {epoch+1}")
    pbar = tqdm(trainloader, total=len(trainloader))
    for batch in pbar:
        optimizer.zero_grad()

        images,texts = batch
        texts = texts.squeeze(1)
        images= images.to(device)
        texts = texts.to(device)

        # print(images.shape, texts.shape)

        # Forward pass
        logits_per_image, logits_per_text = model(images, texts)

        # Compute loss
        ground_truth = torch.arange(BATCH_SIZE).to(device)
        total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2

        # Backward pass
        total_loss.backward()
        if device == "cpu":
            optimizer.step()
        else :
            convert_models_to_fp32(model)
            optimizer.step()
            clip.model.convert_weights(model)

        pbar.set_description(f"Epoch {epoch}/{EPOCH}, Loss: {total_loss.item():.4f}")

In [46]:
from tqdm.notebook import tqdm

# Assuming model, trainloader, valloader, optimizer, and other necessities are already defined
# Ensure the device is set correctly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def compute_accuracy(logits, ground_truth):
    _, predicted = logits.max(1)
    total = ground_truth.size(0)
    correct = (predicted == ground_truth).sum().item()
    return correct / total

for epoch in range(EPOCH):
    print(f"Epoch: {epoch+1}")
    
    # Training loop
    model.train()
    train_total, train_correct = 0, 0
    pbar = tqdm(trainloader, total=len(trainloader))
    for batch in pbar:
        optimizer.zero_grad()
        
        images, texts = batch
        texts = texts.squeeze(1)
        images = images.to(device)
        texts = texts.to(device)

        logits_per_image, logits_per_text = model(images, texts)

        # Compute loss
        ground_truth = torch.arange(BATCH_SIZE).to(device)
        total_loss = (loss_img(logits_per_image, ground_truth) + loss_txt(logits_per_text, ground_truth)) / 2
        
        # Compute train accuracy
        train_correct += (logits_per_image.argmax(dim=1) == ground_truth).float().sum().item()
        train_total += images.size(0)

        total_loss.backward()

        if device == "cpu":
            optimizer.step()
        else :
            convert_models_to_fp32(model)
            optimizer.step()
            clip.model.convert_weights(model)
        
        train_accuracy = 100 * train_correct / train_total
        pbar.set_description(f"Epoch {epoch+1}/{EPOCH}, Loss: {total_loss.item():.4f}, Train Acc: {train_accuracy:.2f}%")

    # Validation loop
    model.eval()
    val_total, val_correct = 0, 0
    with torch.no_grad():
        for batch in valloader:
            images, texts = batch
            texts = texts.squeeze(1)
            images = images.to(device)
            texts = texts.to(device)

            logits_per_image, _ = model(images, texts)
            
            actual_batch_size = logits_per_image.size(0)
            ground_truth = torch.arange(actual_batch_size).to(device)
            # ground_truth = torch.arange(BATCH_SIZE).to(device)

            val_correct += (logits_per_image.argmax(dim=1) == ground_truth).float().sum().item()
            val_total += images.size(0)

    val_accuracy = 100 * val_correct / val_total
    print(f"Validation Accuracy: {val_accuracy:.2f}%")


Epoch: 1


  0%|          | 0/279 [00:00<?, ?it/s]